In [1]:
import pandas as pd
import duckdb
import psycopg2

In [8]:
with open('db_password.txt', 'r') as file:
    db_port = file.readline().strip()
    db_password = file.readline().strip()
    db_name = file.readline().strip()

psql_conn = psycopg2.connect(f"port = '{db_port}' dbname = '{db_name}' user = 'postgres' host = 'localhost' password = '{db_password}'")

table_names = ['item', 'lost_item', 'found_item', 'users', 'locations', 'locates', 'stores', 'returns_or_claims', 'category', 'belongs', 'posts', 'members', 'comments', 'message', 'feedback', 'notifies', 'reward'] 
con = duckdb.connect()

for table_name in table_names:
    query_str = "SELECT * FROM " + table_name
    df = pd.read_sql_query(query_str, psql_conn)
    con.register(table_name, df)

psql_conn.close()

/var/folders/27/fzkmh2w5473fr2qqvscb2bcc0000gn/T/ipykernel_35244/3564826330.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_str, psql_conn)
/var/folders/27/fzkmh2w5473fr2qqvscb2bcc0000gn/T/ipykernel_35244/3564826330.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_str, psql_conn)
/var/folders/27/fzkmh2w5473fr2qqvscb2bcc0000gn/T/ipykernel_35244/3564826330.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_str, psql_conn)
/var/fo

In [55]:
# function that sees SQL command AS input and output table from database
def query(query):
    result = con.execute(query).fetchall()
    column_names = [desc[0] for desc in con.description]
    return column_names, result

In [56]:
# Query row count for each table and store in DataFrame
row_counts = []
for table_name in table_names:
    query_str = f"SELECT COUNT(*) FROM {table_name}"
    result = con.execute(query_str).fetchall()[0][0]
    row_counts.append({'Table Name': table_name, 'Row Count': result})

# Convert to DataFrame
row_counts_df = pd.DataFrame(row_counts)

row_counts_df

,Table Name,Row Count
0,item,1000
1,lost_item,507
2,found_item,493
3,users,3000
4,locations,47
5,locates,1000
6,stores,1000
7,returns_or_claims,10000
8,category,10
9,belongs,1000


In [57]:
columns, data = query('''
SELECT m.senderid, u_s.username as senderName, m.receiverid, u_r.username as receiverName, m.mgtime, mgcontent
from MESSAGE as m
join USERS as u_s on m.senderid = u_s.userid
join USERS as u_r on m.receiverid = u_r.userid
          ''')
message = pd.DataFrame(data, columns=columns)
message

,senderid,senderName,receiverid,receiverName,mgtime,mgcontent
0,UR02638113,葉國芬,UB08270982,武修涵,2024-10-24 06:40:43,感謝好心人找到了我的東西！
1,UB08105600,邵蓉善,UB05811762,韓陽善,2024-10-24 06:40:43,我弄丟了它，請問可以提供更詳細的描述嗎？
2,UD01195700,紀佳俞,UB04210169,滕志綺,2024-10-24 06:45:35,我記得放在椅子上，請問可以再提供一些細節嗎？
3,UB04683037,陳卜俞,UB02795932,東方松恆,2024-10-24 06:46:28,謝謝你，我終於找回來了！
4,UB01802275,沈新軒,UB01625654,杜瑩元,2024-10-24 06:47:17,謝謝你撿到我的耳機，我已經領回來了。
...,...,...,...,...,...,...
29995,UB08034378,上官宜蕙,UB09340643,秦凌豪,2024-11-29 15:57:08,非常感謝，你讓我找回了我的東西，我會盡快來領取！
29996,UR05159676,吳泓如,UB06919894,尤偉廷,2024-11-29 16:20:32,太感謝了！我終於找到了，謝謝你！
29997,UB04911902,程宏雅,UB06560835,紀心翰,2024-11-29 16:26:34,我記得放在椅子上，請問可以再提供一些細節嗎？
29998,UB07050678,劉廷廷,UB09340643,秦凌豪,2024-11-30 07:06:38,這是我弄丟的，請問是在哪個地點找到的呢？


In [63]:
def message(user):
    columns, data = query(f'''
    SELECT 
        CASE 
            WHEN u_s.username = '{user}' THEN u_r.username
            ELSE u_s.username
        END AS the_username,
        MAX(m.mgtime) AS last_message_time
    FROM MESSAGE AS m
    JOIN USERS AS u_s ON m.senderid = u_s.userid
    JOIN USERS AS u_r ON m.receiverid = u_r.userid
    WHERE u_s.username = '{user}' OR u_r.username = '{user}'
    GROUP BY the_username
    ORDER BY last_message_time DESC
    limit 10
    ''') 
    return pd.DataFrame(data, columns=columns)

# 用戶名稱
user = "葉國芬"
result = message(user)

result


,the_username,last_message_time
0,武修涵,2024-11-28 17:43:16
1,司馬潔心,2024-11-26 19:18:02
2,諸葛廷倫,2024-11-24 13:53:05
3,黃婌盈,2024-11-21 15:40:25
4,諸葛軒雯,2024-11-15 13:44:28
5,蘇恆靜,2024-11-15 03:27:53
6,鄭家淑,2024-11-12 05:26:02
7,江鐸思,2024-11-10 12:20:01


In [65]:
def message_people(user, other_user):
    columns, data = query(f'''
    SELECT m.senderid, u_s.username as senderName, m.receiverid, u_r.username as receiverName, m.mgtime, mgcontent
    from MESSAGE as m
    join USERS as u_s on m.senderid = u_s.userid
    join USERS as u_r on m.receiverid = u_r.userid
    where (senderName = '{user}' and receiverName = '{other_user}') or (senderName = '{other_user}' and receiverName = '{user}')
    order by mgtime desc
            ''') 
    return pd.DataFrame(data, columns=columns)

other_user = "武修涵"
message_people(user, other_user)

,senderid,senderName,receiverid,receiverName,mgtime,mgcontent
0,UR02638113,葉國芬,UB08270982,武修涵,2024-11-28 17:43:16,真的很感謝，這個物品對我很重要！
1,UR02638113,葉國芬,UB08270982,武修涵,2024-11-01 18:54:05,非常感謝，你讓我找回了我的東西，我會盡快來領取！
2,UR02638113,葉國芬,UB08270982,武修涵,2024-10-24 06:40:43,感謝好心人找到了我的東西！


In [66]:
def feedback():
    columns, data = query('''
    SELECT f.userid, u.username, f.fbtime, f.fbcontent
    from feedback as f
    join USERS as u on f.userid = u.userid
    ''')
    return pd.DataFrame(data, columns=columns).sort_values("fbtime")

feedback()

,userid,username,fbtime,fbcontent
352,US00001527,None,2023-01-01 14:32:57,這個系統真的是太方便了，謝謝你們幫助我找回了失物。我一定會繼續使用並推薦給更多人！
183,US00000877,None,2023-01-02 21:51:36,最近在提交失物信息時，系統有時會顯示錯誤，無法成功提交，請協助處理一下。
34,US00000102,None,2023-01-04 09:50:42,這個系統真的是太方便了，謝謝你們幫助我找回了失物。我一定會繼續使用並推薦給更多人！
75,US00000337,None,2023-01-04 20:09:54,希望能夠增加一個自動通知功能，當有相關物品被找到時，系統能及時通知我。
175,US00000839,None,2023-01-05 23:28:20,希望能在物品描述中添加照片的上傳功能，這樣能幫助我更快地找到丟失的物品。
...,...,...,...,...
958,UB01619179,郭玲佳,2024-11-25 02:33:45,有時候更新物品狀態的反應速度較慢，希望能加快系統反應時間。
523,UB05893717,辛靈宏,2024-11-26 00:15:44,非常感謝這個系統，讓我找回了丟失的物品！希望大家能夠更積極參與，讓失物招領系統更加完善！
207,UB09674971,宗韋昌,2024-11-26 11:00:13,非常感謝這個系統，讓我找回了丟失的物品！希望大家能夠更積極參與，讓失物招領系統更加完善！
469,US00000128,None,2024-11-28 08:14:31,最近在提交失物信息時，系統有時會顯示錯誤，無法成功提交，請協助處理一下。


In [67]:
columns, data = query('''
SELECT c.memberid, u.username, c.itemid, i.description, c.cmtime, c.cmcontent
from COMMENTS as c
join USERS as u on c.memberid = u.userid
join ITEM as i on c.itemid = i.itemid
''')
# 將結果轉換為 DataFrame
comment = pd.DataFrame(data, columns=columns)
comment

,memberid,username,itemid,description,cmtime,cmcontent
0,UR05795314,曾峰國,IT00000426,環保的收音機,2024-11-05 03:46:47,太感謝了！我終於找到了，謝謝你！
1,UB04250020,高元慧,IT00000289,復古的印章,2024-11-23 01:11:32,這是我弄丟的，請問是在哪個地點找到的呢？
2,UB04513582,許嵩茜,IT00000940,紫色的筆,2024-11-26 23:54:06,太感謝了！我終於找到了，謝謝你！
3,UB07960778,馬艾宇,IT00000627,可愛的電腦,2024-11-13 08:26:45,非常感謝，你讓我找回了我的東西，我會盡快來領取！
4,UR02668503,杜志曉,IT00000898,高科技的梳子,2024-11-14 21:14:53,感謝好心人找到了我的東西！
...,...,...,...,...,...,...
14995,UB08759413,孔峰海,IT00000546,獨特的小刀,2024-11-06 05:14:11,謝謝你的幫忙，已經去領回物品了，感恩！
14996,UB06527110,江淑俊,IT00000834,豪華的毛毯,2024-11-11 02:07:23,我丟了它請問在哪裡找回來？
14997,UB02913030,陸壬淑,IT00000876,紫色的手電筒,2024-11-09 16:50:59,我弄丟了它，請問可以提供更詳細的描述嗎？
14998,UB06177017,崔榆如,IT00000876,紫色的手電筒,2024-11-22 17:54:30,太感謝了！我終於找到了，謝謝你！


In [69]:
def comment(itemID):
    columns, data = query(f'''
    SELECT c.memberid, u.username, c.itemid, i.description, c.cmtime, c.cmcontent
    from COMMENTS as c
    join USERS as u on c.memberid = u.userid
    join ITEM as i on c.itemid = i.itemid
    WHERE i.itemid = '{itemID}'
    ''')
    # 將結果轉換為 DataFrame
    return pd.DataFrame(data, columns=columns).sort_values("cmtime")

itemID = 'IT00000426'
comment(itemID)

,memberid,username,itemid,description,cmtime,cmcontent
8,UB04075553,楊婌光,IT00000426,環保的收音機,2024-10-24 11:25:35,這是我弄丟的，請問是在哪個地點找到的呢？
11,UB01366897,阮澄峰,IT00000426,環保的收音機,2024-10-27 00:42:36,謝謝你撿到！真的太感謝了！
12,UB04137428,皇甫玉哲,IT00000426,環保的收音機,2024-10-28 07:53:41,真的很感謝，這個物品對我很重要！
15,UB05625909,東方舟志,IT00000426,環保的收音機,2024-11-02 12:54:46,我的丟失了，請問這是我的嗎？
1,UR09111902,王艾靜,IT00000426,環保的收音機,2024-11-03 05:58:44,非常感謝，你讓我找回了我的東西，我會盡快來領取！
7,UR05795314,曾峰國,IT00000426,環保的收音機,2024-11-05 03:46:47,太感謝了！我終於找到了，謝謝你！
0,UB09596310,馬潔嵩,IT00000426,環保的收音機,2024-11-08 04:02:03,謝謝你撿到我的物品！我會去指定地點領取。
5,UB07243352,馬惠偉,IT00000426,環保的收音機,2024-11-10 09:27:14,感謝好心人找到了我的東西！
4,UB07997435,梁葦霖,IT00000426,環保的收音機,2024-11-11 21:25:26,我的丟失了，請問這是我的嗎？
14,UB03595108,古祐莉,IT00000426,環保的收音機,2024-11-14 10:22:38,謝謝你撿到我的耳機，我已經領回來了。
